# Comprehensive Analysis: CPU vs GPU Model Training Results

This notebook analyzes the combined results from:
- **CPU Cluster**: 90 traditional ML models
- **GPU Cluster**: 10 PyTorch deep learning models

Results are loaded from the shared Delta table: `ryuta.ray.model_training_results`

## 1. Setup and Imports

In [ ]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Imports successful!")

## 2. Load Results from Delta Table

In [ ]:
# Configuration
RESULTS_TABLE = 'ryuta.ray.model_training_results'

print(f"Loading results from {RESULTS_TABLE}...")

# Load from Delta table
results_spark_df = spark.table(RESULTS_TABLE)

# Convert to pandas
results_df = results_spark_df.toPandas()

print(f"Loaded {len(results_df)} model results")
print(f"\nColumns: {list(results_df.columns)}")
print(f"\nFirst few rows:")
results_df.head()

In [ ]:
# Data summary
print("Dataset Summary")
print("="*80)
print(f"Total models: {len(results_df)}")
print(f"CPU models: {len(results_df[results_df['cluster_type'] == 'cpu'])}")
print(f"GPU models: {len(results_df[results_df['cluster_type'] == 'gpu'])}")
print(f"\nModel types:")
print(results_df['model_type'].value_counts())
print(f"\nFeature strategies:")
print(results_df['feature_strategy'].value_counts())

## 3. Overall Performance Metrics

In [ ]:
print("\n" + "="*80)
print("OVERALL PERFORMANCE METRICS")
print("="*80)

metrics = ['accuracy', 'roc_auc', 'f1', 'precision', 'recall']

print("\nAll Models:")
for metric in metrics:
    print(f"  {metric.upper():12s}: Mean={results_df[metric].mean():.4f}, "
          f"Std={results_df[metric].std():.4f}, "
          f"Max={results_df[metric].max():.4f}, "
          f"Min={results_df[metric].min():.4f}")

print(f"\nTraining Time:")
print(f"  Total: {results_df['training_time'].sum():.2f}s ({results_df['training_time'].sum()/60:.2f} minutes)")
print(f"  Mean: {results_df['training_time'].mean():.2f}s")
print(f"  Median: {results_df['training_time'].median():.2f}s")
print(f"  Max: {results_df['training_time'].max():.2f}s")
print(f"  Min: {results_df['training_time'].min():.2f}s")

## 4. Top Performing Models

In [ ]:
print("\n" + "="*80)
print("TOP 10 MODELS BY ROC AUC")
print("="*80)

top_10 = results_df.nlargest(10, 'roc_auc')

display_cols = ['model_id', 'model_type', 'cluster_type', 'n_features_used', 
                'feature_strategy', 'roc_auc', 'accuracy', 'f1', 'training_time']

print(top_10[display_cols].to_string(index=False))

print("\n" + "="*80)
print("TOP 10 MODELS BY ACCURACY")
print("="*80)

top_10_acc = results_df.nlargest(10, 'accuracy')
print(top_10_acc[display_cols].to_string(index=False))

## 5. Performance by Model Type

In [ ]:
print("\n" + "="*80)
print("PERFORMANCE BY MODEL TYPE")
print("="*80)

model_type_stats = results_df.groupby('model_type').agg({
    'roc_auc': ['mean', 'std', 'min', 'max'],
    'accuracy': ['mean', 'std', 'min', 'max'],
    'f1': ['mean', 'std', 'min', 'max'],
    'training_time': ['mean', 'median', 'sum'],
    'model_id': 'count'
}).round(4)

model_type_stats.columns = ['_'.join(col).strip() for col in model_type_stats.columns.values]
model_type_stats = model_type_stats.rename(columns={'model_id_count': 'count'})

# Sort by mean ROC AUC
model_type_stats = model_type_stats.sort_values('roc_auc_mean', ascending=False)

print(model_type_stats)

# Find best model type
best_model_type = model_type_stats['roc_auc_mean'].idxmax()
print(f"\nBest performing model type (by mean ROC AUC): {best_model_type}")
print(f"Mean ROC AUC: {model_type_stats.loc[best_model_type, 'roc_auc_mean']:.4f}")

## 6. CPU vs GPU Comparison

In [ ]:
print("\n" + "="*80)
print("CPU vs GPU CLUSTER COMPARISON")
print("="*80)

cluster_stats = results_df.groupby('cluster_type').agg({
    'roc_auc': ['mean', 'std', 'max'],
    'accuracy': ['mean', 'std', 'max'],
    'f1': ['mean', 'std', 'max'],
    'training_time': ['mean', 'median', 'sum'],
    'model_id': 'count'
}).round(4)

cluster_stats.columns = ['_'.join(col).strip() for col in cluster_stats.columns.values]
cluster_stats = cluster_stats.rename(columns={'model_id_count': 'count'})

print(cluster_stats)

# Statistical comparison
cpu_models = results_df[results_df['cluster_type'] == 'cpu']
gpu_models = results_df[results_df['cluster_type'] == 'gpu']

print("\nDetailed Comparison:")
print(f"\nCPU Models ({len(cpu_models)}):")
print(f"  Mean ROC AUC: {cpu_models['roc_auc'].mean():.4f} ± {cpu_models['roc_auc'].std():.4f}")
print(f"  Best ROC AUC: {cpu_models['roc_auc'].max():.4f}")
print(f"  Total training time: {cpu_models['training_time'].sum():.2f}s ({cpu_models['training_time'].sum()/60:.2f} min)")

print(f"\nGPU Models ({len(gpu_models)}):")
print(f"  Mean ROC AUC: {gpu_models['roc_auc'].mean():.4f} ± {gpu_models['roc_auc'].std():.4f}")
print(f"  Best ROC AUC: {gpu_models['roc_auc'].max():.4f}")
print(f"  Total training time: {gpu_models['training_time'].sum():.2f}s ({gpu_models['training_time'].sum()/60:.2f} min)")

## 7. Performance by Feature Selection Strategy

In [ ]:
print("\n" + "="*80)
print("PERFORMANCE BY FEATURE SELECTION STRATEGY")
print("="*80)

strategy_stats = results_df.groupby('feature_strategy').agg({
    'roc_auc': ['mean', 'std', 'max'],
    'accuracy': ['mean', 'std', 'max'],
    'n_features_used': 'mean',
    'training_time': 'mean',
    'model_id': 'count'
}).round(4)

strategy_stats.columns = ['_'.join(col).strip() for col in strategy_stats.columns.values]
strategy_stats = strategy_stats.rename(columns={'model_id_count': 'count'})
strategy_stats = strategy_stats.sort_values('roc_auc_mean', ascending=False)

print(strategy_stats)

best_strategy = strategy_stats['roc_auc_mean'].idxmax()
print(f"\nBest feature selection strategy: {best_strategy}")
print(f"Mean ROC AUC: {strategy_stats.loc[best_strategy, 'roc_auc_mean']:.4f}")
print(f"Mean features used: {strategy_stats.loc[best_strategy, 'n_features_used_mean']:.1f}")

## 8. Feature Count vs Performance Analysis

In [ ]:
print("\n" + "="*80)
print("FEATURE COUNT vs PERFORMANCE")
print("="*80)

# Create bins for feature counts
results_df['feature_bin'] = pd.cut(results_df['n_features_used'], 
                                     bins=[0, 25, 50, 75, 100], 
                                     labels=['1-25', '26-50', '51-75', '76-100'])

feature_bin_stats = results_df.groupby('feature_bin').agg({
    'roc_auc': ['mean', 'std', 'count'],
    'accuracy': 'mean',
    'n_features_used': 'mean'
}).round(4)

feature_bin_stats.columns = ['_'.join(col).strip() for col in feature_bin_stats.columns.values]
print(feature_bin_stats)

# Correlation analysis
correlation = results_df[['n_features_used', 'roc_auc']].corr().iloc[0, 1]
print(f"\nCorrelation between number of features and ROC AUC: {correlation:.4f}")

## 9. Training Time Analysis

In [ ]:
print("\n" + "="*80)
print("TRAINING TIME ANALYSIS")
print("="*80)

# Training time by model type
time_by_type = results_df.groupby('model_type')['training_time'].agg([
    'count', 'mean', 'median', 'min', 'max', 'sum'
]).round(2)

time_by_type = time_by_type.sort_values('mean', ascending=False)
print("\nTraining Time by Model Type:")
print(time_by_type)

# Fastest and slowest models
fastest = results_df.nsmallest(5, 'training_time')[['model_id', 'model_type', 'cluster_type', 'training_time', 'roc_auc']]
slowest = results_df.nlargest(5, 'training_time')[['model_id', 'model_type', 'cluster_type', 'training_time', 'roc_auc']]

print("\nFastest 5 Models:")
print(fastest.to_string(index=False))

print("\nSlowest 5 Models:")
print(slowest.to_string(index=False))

# Efficiency metric: ROC AUC per second
results_df['efficiency'] = results_df['roc_auc'] / results_df['training_time']
most_efficient = results_df.nlargest(5, 'efficiency')[['model_id', 'model_type', 'roc_auc', 'training_time', 'efficiency']]

print("\nMost Efficient Models (ROC AUC / Training Time):")
print(most_efficient.to_string(index=False))

## 10. Visualizations

In [ ]:
# ROC AUC distribution by model type
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: ROC AUC by model type
model_order = results_df.groupby('model_type')['roc_auc'].mean().sort_values(ascending=False).index
sns.boxplot(data=results_df, y='model_type', x='roc_auc', order=model_order, ax=axes[0])
axes[0].set_title('ROC AUC Distribution by Model Type', fontsize=14, fontweight='bold')
axes[0].set_xlabel('ROC AUC', fontsize=12)
axes[0].set_ylabel('Model Type', fontsize=12)

# Plot 2: ROC AUC by cluster type
sns.boxplot(data=results_df, x='cluster_type', y='roc_auc', ax=axes[1])
axes[1].set_title('ROC AUC Distribution: CPU vs GPU', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Cluster Type', fontsize=12)
axes[1].set_ylabel('ROC AUC', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# Feature count vs Performance
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: Scatter plot
for cluster in results_df['cluster_type'].unique():
    cluster_data = results_df[results_df['cluster_type'] == cluster]
    axes[0].scatter(cluster_data['n_features_used'], cluster_data['roc_auc'], 
                   label=cluster.upper(), alpha=0.6, s=50)

axes[0].set_xlabel('Number of Features Used', fontsize=12)
axes[0].set_ylabel('ROC AUC', fontsize=12)
axes[0].set_title('Feature Count vs ROC AUC', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Training time by model type
time_data = results_df.groupby('model_type')['training_time'].mean().sort_values()
time_data.plot(kind='barh', ax=axes[1], color='steelblue')
axes[1].set_xlabel('Mean Training Time (seconds)', fontsize=12)
axes[1].set_ylabel('Model Type', fontsize=12)
axes[1].set_title('Average Training Time by Model Type', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Performance by feature selection strategy
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: ROC AUC by feature strategy
strategy_order = results_df.groupby('feature_strategy')['roc_auc'].mean().sort_values(ascending=False).index
sns.boxplot(data=results_df, y='feature_strategy', x='roc_auc', order=strategy_order, ax=axes[0])
axes[0].set_title('ROC AUC by Feature Selection Strategy', fontsize=14, fontweight='bold')
axes[0].set_xlabel('ROC AUC', fontsize=12)
axes[0].set_ylabel('Feature Strategy', fontsize=12)

# Plot 2: Model count by type and cluster
model_counts = results_df.groupby(['cluster_type', 'model_type']).size().unstack(fill_value=0)
model_counts.plot(kind='bar', stacked=True, ax=axes[1], colormap='tab10')
axes[1].set_title('Model Distribution by Cluster and Type', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Cluster Type', fontsize=12)
axes[1].set_ylabel('Number of Models', fontsize=12)
axes[1].legend(title='Model Type', bbox_to_anchor=(1.05, 1), loc='upper left')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=0)

plt.tight_layout()
plt.show()

## 11. Best Model Details

In [ ]:
print("\n" + "="*80)
print("BEST MODEL DETAILS")
print("="*80)

best_model = results_df.loc[results_df['roc_auc'].idxmax()]

print(f"\nBest Overall Model:")
print(f"  Model ID: {best_model['model_id']}")
print(f"  Model Type: {best_model['model_type']}")
print(f"  Cluster Type: {best_model['cluster_type']}")
print(f"  Features Used: {best_model['n_features_used']}")
print(f"  Feature Strategy: {best_model['feature_strategy']}")
print(f"\n  Performance Metrics:")
print(f"    ROC AUC: {best_model['roc_auc']:.4f}")
print(f"    Accuracy: {best_model['accuracy']:.4f}")
print(f"    F1 Score: {best_model['f1']:.4f}")
print(f"    Precision: {best_model['precision']:.4f}")
print(f"    Recall: {best_model['recall']:.4f}")
print(f"\n  Training Details:")
print(f"    Training Time: {best_model['training_time']:.2f} seconds")
print(f"    Device: {best_model.get('device', 'N/A')}")

if 'best_hyperparams' in best_model and pd.notna(best_model['best_hyperparams']):
    try:
        hyperparams = json.loads(best_model['best_hyperparams'])
        print(f"\n  Best Hyperparameters:")
        for param, value in hyperparams.items():
            print(f"    {param}: {value}")
    except:
        pass

In [ ]:
# Best model from each cluster
print("\n" + "="*80)
print("BEST MODELS FROM EACH CLUSTER")
print("="*80)

for cluster in ['cpu', 'gpu']:
    cluster_df = results_df[results_df['cluster_type'] == cluster]
    if len(cluster_df) > 0:
        best_in_cluster = cluster_df.loc[cluster_df['roc_auc'].idxmax()]
        
        print(f"\nBest {cluster.upper()} Model:")
        print(f"  Model ID: {best_in_cluster['model_id']}")
        print(f"  Model Type: {best_in_cluster['model_type']}")
        print(f"  ROC AUC: {best_in_cluster['roc_auc']:.4f}")
        print(f"  Accuracy: {best_in_cluster['accuracy']:.4f}")
        print(f"  Features: {best_in_cluster['n_features_used']} ({best_in_cluster['feature_strategy']})")
        print(f"  Training Time: {best_in_cluster['training_time']:.2f}s")

## 12. Key Insights and Recommendations

In [ ]:
print("\n" + "="*80)
print("KEY INSIGHTS AND RECOMMENDATIONS")
print("="*80)

# Best model type
best_type = results_df.groupby('model_type')['roc_auc'].mean().idxmax()
best_type_score = results_df.groupby('model_type')['roc_auc'].mean().max()

print(f"\n1. Best Model Family:")
print(f"   {best_type} achieved the highest average ROC AUC of {best_type_score:.4f}")

# Best feature strategy
best_strat = results_df.groupby('feature_strategy')['roc_auc'].mean().idxmax()
best_strat_score = results_df.groupby('feature_strategy')['roc_auc'].mean().max()

print(f"\n2. Best Feature Selection Strategy:")
print(f"   '{best_strat}' strategy achieved the highest average ROC AUC of {best_strat_score:.4f}")

# CPU vs GPU comparison
cpu_avg = results_df[results_df['cluster_type'] == 'cpu']['roc_auc'].mean()
gpu_avg = results_df[results_df['cluster_type'] == 'gpu']['roc_auc'].mean()

print(f"\n3. CPU vs GPU Performance:")
if cpu_avg > gpu_avg:
    print(f"   CPU models performed better on average (CPU: {cpu_avg:.4f} vs GPU: {gpu_avg:.4f})")
    print(f"   Traditional ML models may be more suitable for this dataset")
else:
    print(f"   GPU models performed better on average (GPU: {gpu_avg:.4f} vs CPU: {cpu_avg:.4f})")
    print(f"   Deep learning models may capture more complex patterns")

# Feature count insight
corr = results_df[['n_features_used', 'roc_auc']].corr().iloc[0, 1]
print(f"\n4. Feature Count Impact:")
if abs(corr) < 0.1:
    print(f"   Weak correlation ({corr:.3f}) between feature count and performance")
    print(f"   Feature quality matters more than quantity")
elif corr > 0:
    print(f"   Positive correlation ({corr:.3f}) - more features generally help")
else:
    print(f"   Negative correlation ({corr:.3f}) - fewer features may reduce overfitting")

# Training efficiency
fastest_type = results_df.groupby('model_type')['training_time'].mean().idxmin()
fastest_time = results_df.groupby('model_type')['training_time'].mean().min()

print(f"\n5. Training Efficiency:")
print(f"   {fastest_type} was the fastest to train (avg: {fastest_time:.2f}s)")

# Overall recommendation
print(f"\n6. Overall Recommendation:")
print(f"   For best performance: Use {best_type} with '{best_strat}' feature selection")
print(f"   Expected ROC AUC: ~{results_df[(results_df['model_type'] == best_type) & (results_df['feature_strategy'] == best_strat)]['roc_auc'].mean():.4f}")

# Parallel execution benefit
total_time = results_df['training_time'].sum()
print(f"\n7. Parallel Execution Benefit:")
print(f"   Total training time for all models: {total_time:.2f}s ({total_time/60:.2f} minutes)")
print(f"   With Ray parallel execution, this was achieved much faster than sequential training")
print(f"   Estimated sequential time: ~{total_time:.2f}s ({total_time/60:.2f} minutes)")
print(f"   Actual wall-clock time depends on cluster parallelization")

## 13. Export Summary Report

In [ ]:
# Create summary report
summary_report = {
    'total_models': len(results_df),
    'cpu_models': len(results_df[results_df['cluster_type'] == 'cpu']),
    'gpu_models': len(results_df[results_df['cluster_type'] == 'gpu']),
    'best_overall_model_id': int(results_df['roc_auc'].idxmax()),
    'best_roc_auc': float(results_df['roc_auc'].max()),
    'mean_roc_auc': float(results_df['roc_auc'].mean()),
    'best_model_type': best_type,
    'best_feature_strategy': best_strat,
    'total_training_time_seconds': float(results_df['training_time'].sum()),
    'mean_training_time_seconds': float(results_df['training_time'].mean())
}

print("\n" + "="*80)
print("SUMMARY REPORT")
print("="*80)
print(json.dumps(summary_report, indent=2))

# Save summary to Delta table
summary_df = pd.DataFrame([summary_report])
summary_df['analysis_timestamp'] = pd.Timestamp.now()

summary_spark_df = spark.createDataFrame(summary_df)

summary_table = 'ryuta.ray.model_training_summary'
print(f"\nSaving summary report to {summary_table}...")

summary_spark_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(summary_table)

print("Summary report saved successfully!")

## Summary

This notebook provided comprehensive analysis of all 100 models trained across CPU and GPU clusters:

### Analyses Performed:
1. Overall performance metrics across all models
2. Top 10 best performing models identification
3. Performance comparison by model type
4. CPU vs GPU cluster comparison
5. Feature selection strategy analysis
6. Feature count vs performance relationship
7. Training time and efficiency analysis
8. Visual comparisons and distributions
9. Best model detailed breakdown
10. Key insights and recommendations

### Key Takeaways:
- Identified the best performing model families
- Determined optimal feature selection strategies
- Compared CPU vs GPU training effectiveness
- Analyzed the impact of feature count on performance
- Evaluated training efficiency across different models

Use these insights to:
- Select the best model for production deployment
- Optimize feature engineering pipelines
- Choose appropriate compute resources for future training
- Make data-driven decisions about model selection